In [ ]:
#learn more about pyterrier: https://pyterrier.readthedocs.io/en/latest/

In [18]:
import pyterrier as pt
import requests
import pandas as pd
import pickle
import nltk
import numpy as np

In [19]:
data = pd.read_json("./literature.json")

In [20]:
#initialise pyterrier
if not pt.started():
    pt.init()

/tmp/ipykernel_14211/2275133231.py:2: DeprecationWarning: Call to deprecated function (or staticmethod) started. (use pt.java.started() instead) -- Deprecated since version 0.11.0.
  if not pt.started():


In [21]:
#pyterrier exspects a docno field

#data['docno'] = ???
#data['text'] = ???

data['docno'] = data['url']
data['text'] = data['title']

#year field should be string to simplify indexing 
data['year'] = data['year'].astype('str')

In [22]:
#transfrom dataframe into list of dictionaries
data_dict = data.to_dict(orient='records')

In [23]:
index_folder_mult = "./ai_llm_index"

#fields = ???
fields=['docno', 'text', 'authors', 'year', 'title']

indexer_mult = pt.IterDictIndexer(index_folder_mult, meta={'docno': 1024, 'text': 4096, 'year' : 1024}, fields=True, text_attrs=fields, overwrite=True)

index_ref_mult = indexer_mult.index(data_dict, fields=fields)

print(f"path to our index: {index_ref_mult.toString()}")

path to our index: ./ai_llm_index/data.properties


In [24]:
#index_mult = ???
index_mult = pt.IndexFactory.of(index_ref_mult)

In [25]:
print(index_mult.getCollectionStatistics().toString())

Number of documents: 4742
Number of terms: 21695
Number of postings: 106267
Number of fields: 5
Number of tokens: 141709
Field names: [docno, text, authors, year, title]
Positions:   false



In [26]:
search_engine_mult = pt.BatchRetrieve(index_mult, wmodel="TF_IDF")

/tmp/ipykernel_14211/495383578.py:1: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  search_engine_mult = pt.BatchRetrieve(index_mult, wmodel="TF_IDF")


In [27]:
search_engine_mult.search("sentiment analysis")


,qid,docid,docno,rank,score,query
0,1,1694,https://www.ijtsrd.comengineering/computer-eng...,0,11.479152,sentiment analysis
1,1,744,https://www.ijtsrd.comcomputer-science/artific...,1,11.398638,sentiment analysis
2,1,2153,,2,11.013882,sentiment analysis
3,1,1711,https://www.ijtsrd.comcomputer-science/data-pr...,3,11.012437,sentiment analysis
4,1,1219,http://dblp.uni-trier.de/db/conf/ai/ai2011.htm...,4,10.794416,sentiment analysis
...,...,...,...,...,...,...
99,1,3874,https://www.semanticscholar.org/paper/00a40754...,99,3.355589,sentiment analysis
100,1,4306,https://www.semanticscholar.org/paper/bb065603...,100,3.168347,sentiment analysis
101,1,3904,https://www.semanticscholar.org/paper/202f2e0b...,101,3.124757,sentiment analysis
102,1,4508,https://www.semanticscholar.org/paper/68f14172...,102,1.807080,sentiment analysis


In [ ]:
#save our data
#pickle.dump(data, open("./data_exercise.pkl", "wb"))

In [33]:
search_engine = pt.BatchRetrieve(index_mult, wmodel="TF_IDF", metadata=["docno", "year"]) 

#here we add 2 filters chained after the retrieval application
year_filter = pt.apply.generic(lambda res : res[res["year"] >= '2015'])
year_filter2 = pt.apply.generic(lambda res : res[res["year"] <= '2020'])

rank_filter = pt.apply.generic(lambda res : res[res["rank"] < 50])

search_engine_year_filter = search_engine >> rank_filter

search_engine_year_filter = search_engine >> year_filter >> year_filter2 >> rank_filter

/tmp/ipykernel_14211/21346962.py:1: DeprecationWarning: Call to deprecated class BatchRetrieve. (use pt.terrier.Retriever() instead) -- Deprecated since version 0.11.0.
  search_engine = pt.BatchRetrieve(index_mult, wmodel="TF_IDF", metadata=["docno", "year"])


In [34]:
search_engine_year_filter.search("sentiment analysis")

,qid,docid,docno,year,rank,score,query
5,1,1842,http://www.ijtsrd.com/engineering/computer-eng...,2018,5,10.651549,sentiment analysis
7,1,1243,http://dblp.uni-trier.de/db/conf/ai/ai2018.htm...,2018,7,9.997558,sentiment analysis
10,1,1397,http://dblp.uni-trier.de/db/conf/ai/ai2015.htm...,2015,10,5.915099,sentiment analysis
12,1,1448,http://dblp.uni-trier.de/db/conf/ai/ai2017.htm...,2017,12,5.805934,sentiment analysis
14,1,3032,http://dblp.uni-trier.de/db/conf/clic-it/clic-...,2017,14,5.700725,sentiment analysis
16,1,1465,http://dblp.uni-trier.de/db/conf/ai/ai2015.htm...,2015,16,5.599262,sentiment analysis
17,1,1581,http://dblp.uni-trier.de/db/conf/ai/ai2020.htm...,2020,17,5.599262,sentiment analysis
18,1,4607,https://www.semanticscholar.org/paper/5d22b241...,2019,18,5.453662,sentiment analysis
19,1,4538,https://www.semanticscholar.org/paper/6f2ec690...,2017,19,5.315443,sentiment analysis
22,1,232,https://www.ijtsrd.com/computer-science/artifi...,2019,22,4.847041,sentiment analysis
